In [1]:
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import numpy as np
import sys

sys.path.append(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\global-coastal-atlas\notebooks\01. utils')

from plot_funcs import future_shorelinepositions


In [2]:
# load in and keep only sandy transects
path_annual = r'P:\1000545-054-globalbeaches\03_Global_shorelines_update_2020\Transect_Mapping\Version_16102018\CSV\ShorelineMonitor_1984_2021_v1.5_set1.csv'
df_annual = pd.read_csv(path_annual)
df_annual = df_annual[df_annual['flag_sandy'] == True]

df_annual = gpd.GeoDataFrame(df_annual, geometry= gpd.points_from_xy(df_annual.Intersect_lon, df_annual.Intersect_lat)).set_crs('epsg:3587')
df_annual = df_annual.to_crs('+proj=wintri')

df_annual.columns

Index(['transect_id', 'country_id', 'continent', 'country_name', 'changerate',
       'changerate_unc', 'flag_sandy', 'no_shorelines', 'dt', 'dist', 'RMSE',
       'outliers_1', 'outliers_2', 'Timespan', 'intercept', 'intercept_unc',
       'Start_lon', 'Start_lat', 'Intersect_lon', 'Intersect_lat', 'End_lon',
       'End_lat', 'coastline_idint', 'no_sedcomp', 'low_detect_shlines',
       'err_changerate', 'err_timespan', 'geometry'],
      dtype='object')

#### GHG emmisions (Vousdoukas, 2021)

In [3]:
basepath_vous = r'P:\1000545-054-globalbeaches\01_Global_Future_Shorelines\globalErosionProjectionsDataset_linear'

#RCP4.5 2050
df_vous_45_2050 = pd.read_csv(os.path.join(basepath_vous, 'globalErosionProjections_SLR_Retreat_RCP45_2050.csv'), header= None)
df_vous_45_2050 = df_vous_45_2050.rename(columns= {0: 'latitude', 1: 'longitude', 2: 'p1_45_2050', 3: 'p5_45_2050', 
                                                   4: 'p17_45_2050', 5: 'p50_45_2050', 6: 'p83_45_2050', 7: 
                                                   'p95_45_2050', 8: 'p99_45_2050'})
df_vous_45_2050 = df_vous_45_2050.drop_duplicates(subset= ['latitude', 'longitude'], keep = 'first')

#RCP4.5 2100
df_vous_45_2100 = pd.read_csv(os.path.join(basepath_vous, 'globalErosionProjections_SLR_Retreat_RCP45_2100.csv'), header= None)
df_vous_45_2100 = df_vous_45_2100.rename(columns= {0: 'latitude', 1: 'longitude', 2: 'p1_45_2100', 3: 'p5_45_2100', 
                                                   4: 'p17_45_2100', 5: 'p50_45_2100', 6: 'p83_45_2100', 7: 
                                                   'p95_45_2100', 8: 'p99_45_2100'})
df_vous_45_2100 = df_vous_45_2100.drop_duplicates(subset= ['latitude', 'longitude'], keep = 'first')

#RCP8.5 2050
df_vous_85_2050 = pd.read_csv(os.path.join(basepath_vous, 'globalErosionProjections_SLR_Retreat_RCP85_2050.csv'), header= None)
df_vous_85_2050 = df_vous_85_2050.rename(columns= {0: 'latitude', 1: 'longitude', 2: 'p1_85_2050', 3: 'p5_85_2050', 
                                                   4: 'p17_85_2050', 5: 'p50_85_2050', 6: 'p83_85_2050', 7: 
                                                   'p95_85_2050', 8: 'p99_85_2050'})
df_vous_85_2050 = df_vous_85_2050.drop_duplicates(subset= ['latitude', 'longitude'], keep = 'first')

#RCP8.5 2100
df_vous_85_2100 = pd.read_csv(os.path.join(basepath_vous, 'globalErosionProjections_SLR_Retreat_RCP85_2100.csv'), header= None)
df_vous_85_2100 = df_vous_85_2100.rename(columns= {0: 'latitude', 1: 'longitude', 2: 'p1_85_2100', 3: 'p5_85_2100', 
                                                   4: 'p17_85_2100', 5: 'p50_85_2100', 6: 'p83_85_2100', 7: 
                                                   'p95_85_2100', 8: 'p99_85_2100'})
df_vous_85_2100 = df_vous_85_2100.drop_duplicates(subset= ['latitude', 'longitude'], keep = 'first')

df_vous_45_2050.head(2)

,latitude,longitude,p1_45_2050,p5_45_2050,p17_45_2050,p50_45_2050,p83_45_2050,p95_45_2050,p99_45_2050
0,-50.378,-74.386,-2.0775,-1.6823,-1.2463,-0.744,-0.33039,-0.18465,-0.13225
1,-50.386,-74.303,-2.0773,-1.6899,-1.2751,-0.744,-0.34310,-0.18249,-0.12239


In [4]:
[set(df_vous_45_2050['latitude']).difference(x) for x in [set(df_vous_45_2100['latitude']), set(df_vous_85_2050['latitude']), set(df_vous_85_2100['latitude'])]]


[set(), set(), set()]

In [5]:
dfs_vous = [df_vous_45_2100, df_vous_85_2050, df_vous_85_2100]

df_vous = df_vous_45_2050.copy()
for dfv in dfs_vous:
    df_vous = df_vous.merge(dfv, on = ['latitude', 'longitude'], how = 'left')

assert len(df_vous_45_2050) == len(df_vous)

In [6]:
gdf_vous = gpd.GeoDataFrame(df_vous, geometry= gpd.points_from_xy(df_vous.longitude, df_vous_85_2100.latitude)).set_crs('epsg:3587')
gdf_vous = gdf_vous.to_crs('+proj=wintri')

##### Spatial Join on nearest transect

In [7]:
gdf_vous_join = df_annual.sjoin_nearest(gdf_vous, how = 'left', max_distance = 1000)
gdf_vous_join.head(2)


,transect_id,country_id,continent,country_name,changerate,changerate_unc,flag_sandy,no_shorelines,dt,dist,...,p83_85_2050,p95_85_2050,p99_85_2050,p1_85_2100,p5_85_2100,p17_85_2100,p50_85_2100,p83_85_2100,p95_85_2100,p99_85_2100
0,BOX_028_183_0,CHL,South America,Chile,14.088576,1.503763,True,8.0,"[14.0, 16.0, 19.0, 20.0, 22.0, 30.0, 32.0, 35.0]","[242.54210813082503, 300.2922429940901, 346.20...",...,-0.43175,-0.18851,-0.13284,-10.256,-7.5430,-5.0821,-3.0889,-1.3752,-0.5325,-0.33707
92,BOX_028_183_92,CHL,South America,Chile,0.268190,0.211186,True,23.0,"[0.0, 2.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0,...","[497.26072058508424, 515.0110199065714, 491.85...",...,-0.41085,-0.19593,-0.11579,-10.336,-7.6664,-5.1196,-3.0889,-1.2488,-0.5106,-0.25198


In [8]:
def future_timeseries(a, b, cor_2050, cor_2100):
    a = 0
    #dt = np.arange(1984, 2101, 1)

    y2021 = a + (2021 - 1984) * b
    y2050 = a + (2050 - 1984) * b + cor_2050
    y2100 = a + (2100 - 1984) * b + cor_2100

    #s = pd.Series(index = dt, dtype='float64')
    # s.loc[1984] = a
    # s.loc[2021] = y2021
    # s.loc[2050] = y2050
    # s.loc[2100] = y2100

    y = [a, y2021, y2050, y2100]
    #s = s.interpolate(method = 'index').values

    return y

In [9]:
# AMBIENT
gdf_vous_join['Ambient'] = gdf_vous_join.apply(lambda row: future_timeseries(row.intercept, row.changerate, 0, 0), axis = 1)

# RCP4.5
gdf_vous_join['RCP4.5_p5'] = gdf_vous_join.apply(lambda row: future_timeseries(row.intercept, row.changerate, row.p5_45_2050, row.p5_45_2100), axis = 1)
gdf_vous_join['RCP4.5_p50'] = gdf_vous_join.apply(lambda row: future_timeseries(row.intercept, row.changerate, row.p50_45_2050, row.p50_45_2100), axis = 1)
gdf_vous_join['RCP4.5_p95'] = gdf_vous_join.apply(lambda row: future_timeseries(row.intercept, row.changerate, row.p95_45_2050, row.p95_45_2100), axis = 1)

# RCP8.5
gdf_vous_join['RCP8.5_p5'] = gdf_vous_join.apply(lambda row: future_timeseries(row.intercept, row.changerate, row.p5_85_2050, row.p5_85_2100), axis = 1)
gdf_vous_join['RCP8.5_p50'] = gdf_vous_join.apply(lambda row: future_timeseries(row.intercept, row.changerate, row.p50_85_2050, row.p50_85_2100), axis = 1)
gdf_vous_join['RCP8.5_p95'] = gdf_vous_join.apply(lambda row: future_timeseries(row.intercept, row.changerate, row.p95_85_2050, row.p95_85_2100), axis = 1)

In [10]:
gdf_vous_join.head(2)

,transect_id,country_id,continent,country_name,changerate,changerate_unc,flag_sandy,no_shorelines,dt,dist,...,p83_85_2100,p95_85_2100,p99_85_2100,Ambient,RCP4.5_p5,RCP4.5_p50,RCP4.5_p95,RCP8.5_p5,RCP8.5_p50,RCP8.5_p95
0,BOX_028_183_0,CHL,South America,Chile,14.088576,1.503763,True,8.0,"[14.0, 16.0, 19.0, 20.0, 22.0, 30.0, 32.0, 35.0]","[242.54210813082503, 300.2922429940901, 346.20...",...,-1.3752,-0.5325,-0.33707,"[0, 521.2772969558748, 929.8459891645335, 1634...","[0, 521.2772969558748, 928.1636891645335, 1630...","[0, 521.2772969558748, 929.1019891645335, 1632...","[0, 521.2772969558748, 929.6613391645335, 1633...","[0, 521.2772969558748, 927.8591891645335, 1626...","[0, 521.2772969558748, 928.9111091645335, 1631...","[0, 521.2772969558748, 929.6574791645336, 1633..."
92,BOX_028_183_92,CHL,South America,Chile,0.268190,0.211186,True,23.0,"[0.0, 2.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0,...","[497.26072058508424, 515.0110199065714, 491.85...",...,-1.2488,-0.5106,-0.25198,"[0, 9.923013636010973, 17.700510810181736, 31....","[0, 9.923013636010973, 16.010610810181735, 26....","[0, 9.923013636010973, 16.956510810181737, 29....","[0, 9.923013636010973, 17.518020810181735, 30....","[0, 9.923013636010973, 15.752910810181737, 23....","[0, 9.923013636010973, 16.765630810181737, 28....","[0, 9.923013636010973, 17.504580810181736, 30...."


In [11]:
gdf_vous_join.columns

Index(['transect_id', 'country_id', 'continent', 'country_name', 'changerate',
       'changerate_unc', 'flag_sandy', 'no_shorelines', 'dt', 'dist', 'RMSE',
       'outliers_1', 'outliers_2', 'Timespan', 'intercept', 'intercept_unc',
       'Start_lon', 'Start_lat', 'Intersect_lon', 'Intersect_lat', 'End_lon',
       'End_lat', 'coastline_idint', 'no_sedcomp', 'low_detect_shlines',
       'err_changerate', 'err_timespan', 'geometry', 'index_right', 'latitude',
       'longitude', 'p1_45_2050', 'p5_45_2050', 'p17_45_2050', 'p50_45_2050',
       'p83_45_2050', 'p95_45_2050', 'p99_45_2050', 'p1_45_2100', 'p5_45_2100',
       'p17_45_2100', 'p50_45_2100', 'p83_45_2100', 'p95_45_2100',
       'p99_45_2100', 'p1_85_2050', 'p5_85_2050', 'p17_85_2050', 'p50_85_2050',
       'p83_85_2050', 'p95_85_2050', 'p99_85_2050', 'p1_85_2100', 'p5_85_2100',
       'p17_85_2100', 'p50_85_2100', 'p83_85_2100', 'p95_85_2100',
       'p99_85_2100', 'Ambient', 'RCP4.5_p5', 'RCP4.5_p50', 'RCP4.5_p95',
       'R

In [12]:
# gdf_vous_join = gdf_vous_join[['transect_id', 'country_id', 'continent', 'country_name',
#                                 'Ambient', 'RCP4.5_p5', 'RCP4.5_p50', 'RCP4.5_p95', 'RCP8.5_p5', 
#                                 'RCP8.5_p50', 'RCP8.5_p95']]

In [13]:
gdf_vous_join

,transect_id,country_id,continent,country_name,changerate,changerate_unc,flag_sandy,no_shorelines,dt,dist,...,p83_85_2100,p95_85_2100,p99_85_2100,Ambient,RCP4.5_p5,RCP4.5_p50,RCP4.5_p95,RCP8.5_p5,RCP8.5_p50,RCP8.5_p95
0,BOX_028_183_0,CHL,South America,Chile,14.088576,1.503763,True,8.0,"[14.0, 16.0, 19.0, 20.0, 22.0, 30.0, 32.0, 35.0]","[242.54210813082503, 300.2922429940901, 346.20...",...,-1.3752,-0.53250,-0.33707,"[0, 521.2772969558748, 929.8459891645335, 1634...","[0, 521.2772969558748, 928.1636891645335, 1630...","[0, 521.2772969558748, 929.1019891645335, 1632...","[0, 521.2772969558748, 929.6613391645335, 1633...","[0, 521.2772969558748, 927.8591891645335, 1626...","[0, 521.2772969558748, 928.9111091645335, 1631...","[0, 521.2772969558748, 929.6574791645336, 1633..."
92,BOX_028_183_92,CHL,South America,Chile,0.268190,0.211186,True,23.0,"[0.0, 2.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0,...","[497.26072058508424, 515.0110199065714, 491.85...",...,-1.2488,-0.51060,-0.25198,"[0, 9.923013636010973, 17.700510810181736, 31....","[0, 9.923013636010973, 16.010610810181735, 26....","[0, 9.923013636010973, 16.956510810181737, 29....","[0, 9.923013636010973, 17.518020810181735, 30....","[0, 9.923013636010973, 15.752910810181737, 23....","[0, 9.923013636010973, 16.765630810181737, 28....","[0, 9.923013636010973, 17.504580810181736, 30...."
167,BOX_028_183_167,CHL,South America,Chile,0.174932,0.215829,True,26.0,"[0.0, 2.0, 3.0, 13.0, 14.0, 16.0, 17.0, 18.0, ...","[1177.8557098851159, 1172.507897156761, 1145.0...",...,-1.3419,-0.55294,-0.32058,"[0, 6.472476757604102, 11.54549908113164, 20.2...","[0, 6.472476757604102, 9.93959908113164, 16.28...","[0, 6.472476757604102, 10.80149908113164, 18.4...","[0, 6.472476757604102, 11.354499081131639, 19....","[0, 6.472476757604102, 9.546499081131639, 12.5...","[0, 6.472476757604102, 10.61062908113164, 17.2...","[0, 6.472476757604102, 11.336059081131639, 19...."
227,BOX_028_183_227,CHL,South America,Chile,0.073707,0.137905,True,18.0,"[0.0, 2.0, 14.0, 16.0, 17.0, 18.0, 21.0, 22.0,...","[663.5872380921787, 640.9231529697731, 636.764...",...,-99.4580,-59.76200,-44.52400,"[0, 2.727145474595325, 4.864637873602471, 8.54...","[0, 2.727145474595325, -87.90436212639753, -22...","[0, 2.727145474595325, -39.77536212639753, -10...","[0, 2.727145474595325, -10.745362126397527, -2...","[0, 2.727145474595325, -103.60536212639752, -4...","[0, 2.727145474595325, -51.228362126397535, -1...","[0, 2.727145474595325, -14.36336212639753, -51..."
228,BOX_028_183_228,CHL,South America,Chile,0.103907,0.198709,True,23.0,"[0.0, 2.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0,...","[617.288998904033, 608.8196241588425, 600.7101...",...,-99.4580,-59.76200,-44.52400,"[0, 3.8445445286552458, 6.857836186249898, 12....","[0, 3.8445445286552458, -85.91116381375011, -2...","[0, 3.8445445286552458, -37.7821638137501, -99...","[0, 3.8445445286552458, -8.752163813750101, -2...","[0, 3.8445445286552458, -101.6121638137501, -4...","[0, 3.8445445286552458, -49.235163813750106, -...","[0, 3.8445445286552458, -12.370163813750104, -..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1837176,BOX_211_067_127,RUS,Europe,Russia,-999.000000,-999.000000,True,0.0,[],[],...,-112.1100,-62.10000,-50.43500,"[0, -36963.0, -65934.0, -115884.0]","[0, -36963.0, -66002.77, -116036.94]","[0, -36963.0, -65969.01, -115950.982]","[0, -36963.0, -65946.07, -115907.551]","[0, -36963.0, -66079.4, -116406.37]","[0, -36963.0, -66015.27, -116104.21]","[0, -36963.0, -65958.416, -115946.1]"
1837177,BOX_211_067_128,RUS,Europe,Russia,-999.000000,-999.000000,True,0.0,[],[],...,-112.1100,-62.10000,-50.43500,"[0, -36963.0, -65934.0, -115884.0]","[0, -36963.0, -66002.77, -116036.94]","[0, -36963.0, -65969.01, -115950.982]","[0, -36963.0, -65946.07, -115907.551]","[0, -36963.0, -66079.4, -116406.37]","[0, -36963.0, -66015.27, -116104.21]","[0, -36963.0, -65958.416, -115946.1]"
1837178,BOX_211_067_129,RUS,Europe,Russia,-999.000000,-999.000000,True,

In [21]:
gdf_vous_filt = gdf_vous_join[['transect_id', 'country_id', 'continent', 'country_name', 'changerate', 'intercept',
       'Start_lon', 'Start_lat', 'Intersect_lon', 'Intersect_lat', 'End_lon',
       'End_lat',  'Ambient', 'RCP4.5_p5', 'RCP4.5_p50', 'RCP4.5_p95',
       'RCP8.5_p5', 'RCP8.5_p50', 'RCP8.5_p95']]

In [22]:
savefile = r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\datasets\03_Shorelinemonitor_future\future_shorelines.csv'

gdf_vous_filt.to_csv(savefile, index = False)

In [15]:
import pandas as pd
df_fut = pd.read_csv(savefile)

In [16]:
box = 'BOX_187_084_11'

future_shorelinepositions(df_fut = df_fut, df_annual = df_annual, box = box)

In [17]:
box = 'BOX_187_084_11'

future_shorelinepositions(df_fut = df_fut, df_annual = df_annual, box = box)

In [18]:
box = 'BOX_187_090_49'

future_shorelinepositions(df_fut = df_fut, df_annual = df_annual, box = box)